In [35]:
# Inputs from the student
using DataFrames, CSV

# current student using the program
student = 3

# classes the student intends to take
required = []

# student intends to take subject s in semester t
subject = []
semester = []

# a list of (s,t): subject s students canNOT take in semester t
req_not = []

# preferences of workload: # of units per sem, # of hours per sem
preference = []



Any[]

In [36]:
# Importing the courses the student has already taken
using DataFrames, CSV

example_students = CSV.read("ExampleStudents.csv", DataFrame)
example_students

,Student number,Major,Semester 1,Semester 2
,Int64,String,String,String?
1,1,18,"[HASS1, 18.01, 8.01, 5.111]",missing
2,2,18,"[HASS1, 18.01, 8.01, 7.012 ]","[HASS2, 18.02, 18.600, 8.02 ]"
3,3,18,"[HASS1, 18.01, 7.012, 5.111]","[HASS2, 18.02, 18.600, 8.02]"
4,4,15-2:6,"[6.0001/6.0002, 8.01, HASS1,18.01]",missing
5,5,15-2:6,"[6.0001/6.0002, 3.091, HASS1, 18.01]","[8.02, 18.02, 7.012, HASS2]"
6,6,15-2:6,"[HASS1, 6.0001/6.0002, 18.01, 8.01]","[HASS2, 15.053, 8.02, 18.02 ]"


In [75]:
# courses already taken by the student 
completed_courses = []
for i in 3:10
    if typeof(example_students[student, i]) != Missing
        semester  = replace(replace(example_students[student, i], "["=>""), "]"=>"") #remove "[" and "]"" from the string
        semester = replace(semester, " "=>"") #remove spaces from string
        semester = split(semester, ",") #split the string into individual classes
        append!(completed_courses, semester)
    end
end
println(completed_courses)

# compute the student's current term
completed_terms = 0
for i in 3:10
    if typeof(example_students[student, i]) != Missing
        completed_terms += 1
    end
end
current_term = 1 + completed_terms

Any["HASS1", "18.01", "7.012", "5.111", "HASS2", "18.02", "18.600", "8.02", "HASS3", "18.032", "18.650", "18.330", "HASS4", "18.112", "18.212", "18.453", "HASS5", "18.200", "18.410", "18.303"]


6

In [76]:
# Importing all the courses at MIT
using DataFrames, CSV

# all available courses
courses = CSV.read("courses.csv", DataFrame)
courses


,Column1,ID,Units,Course,Prereqs
,Int64,String,Int64,String,String?
1,0,1.00,12,1,GIR:CAL1
2,1,1.001,12,1,GIR:CAL1
3,2,1.004,12,1,missing
4,3,1.006,12,1,missing
5,4,1.013,6,1,Permission of instructor
6,5,1.015,12,1,"2.003, 2.016, and 2.678; [2.671]"
7,6,1.016,9,1,missing
8,7,1.020,12,1,"GIR:PHY1, 18.03, and (1.00 or 1.000)"
9,8,1.021,12,1,"18.03, 3.016B, or permission of instructor"


In [77]:
# --- Model specification
using JuMP, DataFrames, Gurobi

# Model
model = Model(with_optimizer(Gurobi.Optimizer))

# Decision variables
# x(s,t): subject s is taken in semester t
@variable(model, x[1:2292, 1:8], Bin)

# Objective Function
# Maximize the total utility of the subjects taken
@objective(model, Max, sum(courses[i, 6]*x[i, j] for i in 1:2292, j in current_term:8))

# Constraints
# Every GIR requirement = 1, sum = 6   
@constraint(model, sum(x[745, t] for t in 1:8) == 1) #must take 18.01
@constraint(model, sum(x[746, t] + x[747, t] for t in 1:8) == 1) #must take 18.02
@constraint(model, sum(x[1760, t] + x[1761, t] for t in 1:8) == 1) #must take 7.014 or 7.016
@constraint(model, sum(x[1842, t] for t in 1:8) == 1) #must take 8.011 or 8.01
@constraint(model, sum(x[1843, t] + x[1844, t] for t in 1:8) == 1) #must take 8.02 or 8.022
@constraint(model, sum(x[1587, t] + x[1404, t] for t in 1:8) == 1) #must take 5.111 or 3.091

# sum of HASS classes >= 8
sum(isHass[i]*x[i, t] for i in 1:2292, t in 1:8  >= 8 )

# No two subjects taken in Spring 2021 overlap in time

# No subject is taken prior to its prereq

# Student takes 216 units beyond GIRs
sum(x[i, t]*courses[i, 3] - 72 for i in 1:2292, t in 1:8 >= 216)

# Students cannot take the same subject twice (for dataframe x: rowsum <= one)
for i in 1:2292
    sum(x[i,t] for t in 1:8 == 1)
end

# Required, Req, ReqNot specification from student


# Maximum and Minimum number of units taken per semester

# Soft Constraints (Goal Programming)

# Preferred number of units for each semester

# Preferred workload (hrs/week)


LoadError: [91mGurobi Error 10009: Failed to obtain a valid license[39m

In [ ]:
#